In [5]:
!pip install seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import os

# 1. Load dữ liệu và xử lý NaN
def load_data(path):
    df = pd.read_excel(path)
    grouped = df.groupby("index")
    sentences = grouped["tokens"].apply(list).tolist()
    labels = grouped["tags"].apply(list).tolist()
    return sentences[1:], labels[1:]  # bỏ dòng đầu nếu cần
data_path = "/kaggle/input/relabel-cs221"

sentences, labels = None, None
for f in os.listdir(data_path):
    f_path = os.path.join(data_path, f)
    if not sentences or not labels:
        sentences, labels = load_data(f_path)
    sents, labs = load_data(f_path)
    sentences += sents
    labels += labs

# sentences, labels = load_data("/kaggle/input/relabel-cs221/val_500_0.xlsx")
# sents2, labels2 = load_data("/kaggle/input/relabel-cs221/test_500_0.xlsx")
# sents3, labels3 = load_data("/kaggle/input/relabel-cs221/val_500_vert_1.xlsx")
# sents4, labels4 = load_data("/kaggle/input/relabel-cs221/test_500_vert_1.xlsx")

# sentences += sents2 + sents3 + sents4
# labels += labels2 + labels3 + labels4


def contains_nan(seq):
    return any(pd.isna(token) for token in seq)

filtered_sentences, filtered_labels = [], []
for tokens, tags in zip(sentences, labels):
    if not contains_nan(tokens) and not contains_nan(tags):
        filtered_sentences.append(tokens)
        filtered_labels.append(tags)

sentences = filtered_sentences
labels = filtered_labels

# 2. Chuẩn bị tokenizer và Dataset
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
label_list = ["O", "B-PER", "I-PER",  "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

In [3]:
from torch.utils.data import Dataset
import torch

class NERDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len=128):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        words = self.sentences[idx]
        word_labels = self.labels[idx]

        tokens = []
        label_ids = []

        for word, label in zip(words, word_labels):
            word_tokens = self.tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            label_ids.extend([label] + [-100] * (len(word_tokens) - 1))

        tokens = [self.tokenizer.cls_token] + tokens[:self.max_len - 2] + [self.tokenizer.sep_token]
        label_ids = [-100] + label_ids[:self.max_len - 2] + [-100]

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)

        pad_len = self.max_len - len(input_ids)
        input_ids += [self.tokenizer.pad_token_id] * pad_len
        attention_mask += [0] * pad_len
        label_ids += [-100] * pad_len

        return {
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(attention_mask),
            'labels': torch.tensor(label_ids)
        }

In [6]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    true_preds = []

    for pred, label in zip(predictions, labels):
        temp_labels = []
        temp_preds = []
        for p_, l_ in zip(pred, label):
            if l_ != -100:
                temp_labels.append(id2label[l_])
                temp_preds.append(id2label[p_])
        true_labels.append(temp_labels)
        true_preds.append(temp_preds)

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
    }

In [7]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments

model_path = "/kaggle/input/phobert_ner_cs221/other/default/1/phobert-ner/checkpoint-3125"

import json
with open(f"{model_path}/config.json") as f:
    checkpoint_config = json.load(f)

print("Label mapping từ checkpoint:", checkpoint_config["id2label"])

# 3. Sử dụng label mapping từ checkpoint thay vì định nghĩa mới
id2label = {int(k): v for k, v in checkpoint_config["id2label"].items()}
label2id = {v: k for k, v in id2label.items()}
label_list = list(label2id.keys())

model = AutoModelForTokenClassification.from_pretrained(
    # "vinai/phobert-base",
    model_path,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)


sentences = [[str(w) for w in sent] for sent in sentences]

train_dataset = NERDataset(sentences, labels, tokenizer)



args = TrainingArguments(
    output_dir="./phobert-ner",
    per_device_train_batch_size=16,
    num_train_epochs=20,           # Tổng số epoch mong muốn
    logging_dir="./logs",         # 👈 để Trainer log đúng
    logging_steps=100,
    save_strategy="epoch",
    report_to="none",
    save_total_limit=1,
)


# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

# Fine-tune tiếp
trainer.train()

Label mapping từ checkpoint: {'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', '5': 'B-LOC', '6': 'I-LOC'}


/tmp/ipykernel_35/270240647.py:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,0.351000
200,0.106400
300,0.078100
400,0.040400
500,0.027500
600,0.021400
700,0.014700
800,0.011600
900,0.012200
1000,0.007600


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=1520, training_loss=0.04563206637180165, metrics={'train_runtime': 797.1528, 'train_samples_per_second': 60.766, 'train_steps_per_second': 1.907, 'total_flos': 3164446765455360.0, 'train_loss': 0.04563206637180165, 'epoch': 20.0})

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_sentences = [
    ["Thành", "phố", "Hồ", "Chí", "Minh", "là", "trung", "tâm", "kinh", "tế", "lớn", "nhất", "Việt", "Nam", "."],
    ["Tiến", "sĩ", "Lê", "Thị", "Thu", "Hà", "đang", "nghiên", "cứu", "về", "trí", "tuệ", "nhân", "tạo", "."],
    ["Tập", "đoàn", "FPT", "Software", "là", "một", "trong", "những", "công", "ty", "công", "nghệ", "hàng", "đầu", "Việt", "Nam", "."],
    ["Vịnh", "Hạ", "Long", "được", "UNESCO", "công", "nhận", "là", "Di", "sản", "Thiên", "nhiên", "Thế", "giới", "."],
    ["Công", "ty", "cổ", "phần", "Sữa", "Việt", "Nam", "Vinamilk", "là", "nhà", "sản", "xuất", "sữa", "hàng", "đầu", "Việt", "Nam", "."],
    ["Sông", "Hồng", "chảy", "qua", "thủ", "đô", "Hà", "Nội", "của", "Việt", "Nam", "."],
    ["Ông", "Nguyễn", "Thanh", "Long", "từng", "là", "Bộ", "trưởng", "Bộ", "Y", "tế", "."],
    ["Cầu", "Cần", "Thơ", "nối", "liền", "hai", "bờ", "sông", "Hậu", "thuộc", "thành", "phố", "Cần", "Thơ", "."],
    ["Tổng", "Bí", "thư", "Tô", "Lâm", "đã", "có", "bài", "phát", "biểu", "quan", "trọng", "."],
    ["Liên", "Hợp", "Quốc", "có", "trụ", "sở", "chính", "tại", "New", "York", ",", "Hoa", "Kỳ", "."],
    ['đổi', 'Fukui', '(', 'thành', 'phố', ')'],
    ['Con', 'đường', 'tơ', 'lụa'],
    ['đổi', 'Chỉ', 'thị', 'về', 'hạn', 'chế', 'các', 'chất', 'nguy', 'hiểm'],
    ['Tổng', 'thống', 'Hoa', 'Kỳ']
]
token_list = []
label_list = []

for test_sentence in test_sentences:
    model.eval()
    encoding = tokenizer(test_sentence, is_split_into_words=True, return_tensors="pt", truncation=True)
    encoding = {k: v.to(device) for k, v in encoding.items()}
    with torch.no_grad():
        output = model(**encoding)
    
    pred_ids = output.logits.argmax(dim=-1).squeeze().tolist()
    tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"].squeeze())
    
    # Map index về nhãn
    pred_labels = [model.config.id2label[idx] for idx in pred_ids]
    token_list.append(tokens)
    label_list.append(pred_labels)


# **Model chưa finetune predict**

In [26]:
model_ = AutoModelForTokenClassification.from_pretrained(
    "/kaggle/input/phobert_ner_cs221/other/default/1/phobert-ner/checkpoint-3125", 
    num_labels=7
)

label_list_ = []


for test_sentence in test_sentences:
    model_.eval()
    encoding = tokenizer(test_sentence, is_split_into_words=True, return_tensors="pt", truncation=True)
    with torch.no_grad():
        output = model_(**encoding)
    
    pred_ids = output.logits.argmax(dim=-1).squeeze().tolist()
    tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"].squeeze())
    
    # Map index về nhãn
    pred_labels = [model.config.id2label[idx] for idx in pred_ids]
    label_list_.append(pred_labels)

In [27]:
# connect separated tokens
def connect_subwords(token_list, labels1, labels2):
    token_list_new, labels1_new, labels2_new = [], [], []
    for i, tokens in enumerate(token_list):
        t_tokens, t_l1, t_l2 = [], [], []
        flag = 0
        for token, l1, l2 in zip(token_list[i], labels1[i], labels2[i]):
            if token == "<s>" or token == "</s>":
                continue
            if flag == 1:
                flag = 0
                token = token.replace("@", "")
                t_tokens[-1] = t_tokens[-1] + token
                print(t_tokens[-1])
                continue
            if "@" in token and flag == 0:
                flag = 1
                token = token.replace("@", "")
            
            t_tokens.append(token)
            t_l1.append(l1)
            t_l2.append(l2)
        token_list_new.append(t_tokens)
        labels1_new.append(t_l1)
        labels2_new.append(t_l2)
    return token_list_new, labels1_new, labels2_new

token_list, label_list, label_list_ = connect_subwords(token_list, label_list, label_list_)

tuệ
Fukui
thống


In [28]:
for i, token in enumerate(token_list):
    if i not in [4, 8, 11]:
        continue
    for token, label, label_, in zip(token_list[i], label_list[i], label_list_[i]):
        print(f"{token:10} {label:8} {label_:8}")
        # print(f"{token:10} {label_:8}")
    print("="*50)

Công       B-ORG    O       
ty         I-ORG    O       
cổ         I-ORG    I-ORG   
phần       I-ORG    I-ORG   
Sữa        I-ORG    I-ORG   
Việt       I-ORG    I-ORG   
Nam        I-ORG    I-ORG   
Vinamilk   I-ORG    B-ORG   
là         O        O       
nhà        O        O       
sản        O        O       
xuất       O        O       
sữa        O        O       
hàng       O        O       
đầu        O        O       
Việt       B-LOC    B-LOC   
Nam        I-LOC    I-LOC   
.          O        O       
Tổng       B-PER    B-ORG   
Bí         I-PER    I-ORG   
thư        I-PER    I-ORG   
Tô         I-PER    I-ORG   
Lâm        I-PER    I-ORG   
đã         O        O       
có         O        O       
bài        O        O       
phát       O        O       
biểu       O        O       
quan       O        O       
trọng      O        O       
.          O        O       
Con        B-LOC    B-ORG   
đường      I-LOC    I-ORG   
tơ         I-LOC    I-ORG   
lụa        I-L

In [30]:
# Print
for i, tokens in enumerate(token_list):
    tokens_ = []
    labels = []
    labels_ = []
    
    for token, label, label_ in zip(token_list[i], label_list[i], label_list_[i]):
        # label = label2id[label]
        # label_ = label2id[label_]
        pad = 6
        if len(token) > 5:
            pad = 7
        
        tokens_.append(f"{token:{pad}}")
        labels.append(f"{label:{pad}}")
        labels_.append(f"{label_:{pad}}")

    print("Tokens:    " + " ".join(tokens_))
    print("PhoBERT:   " + " ".join(labels_))
    print("Finetuned: " + " ".join(labels))
    print("=" * 100)

output_lines = []

for i, tokens in enumerate(token_list):
    tokens_ = []
    labels = []
    labels_ = []

    for token, label, label_ in zip(token_list[i], label_list[i], label_list_[i]):
        pad = 6 
        if len(token) > 5:
            pad = len(token)
        
        tokens_.append(f"{token:{pad}}")
        labels.append(f"{label:{pad}}")
        labels_.append(f"{label_:{pad}}")

    line1 = "Tokens:    " + " ".join(tokens_)
    line2 = "PhoBERT:   " + " ".join(labels_)
    line3 = "Finetuned: " + " ".join(labels)
    separator = "=" * 100

    output_lines.extend([line1, line2, line3, separator])

# Ghi ra file
with open("ner_comparison.txt", "w", encoding="utf-8") as f:
    for line in output_lines:
        f.write(line + "\n")



Tokens:    Thành  phố    Hồ     Chí    Minh   là     trung  tâm    kinh   tế     lớn    nhất   Việt   Nam    .     
PhoBERT:   B-LOC  I-LOC  I-LOC  I-LOC  I-LOC  O      O      I-LOC  I-LOC  I-LOC  O      O      B-LOC  I-LOC  O     
Finetuned: B-LOC  I-LOC  I-LOC  I-LOC  I-LOC  O      O      I-LOC  I-LOC  O      O      O      B-LOC  I-LOC  O     
Tokens:    Tiến   sĩ     Lê     Thị    Thu    Hà     đang   nghiên  cứu    về     trí    tuệ    nhân   tạo    .     
PhoBERT:   O      O      B-PER  I-PER  I-PER  I-PER  O      O       O      O      O      O      O      O      O     
Finetuned: O      O      B-PER  I-PER  I-PER  I-PER  O      O       O      O      O      O      O      O      O     
Tokens:    Tập    đoàn   FPT    Software là     một    trong  những  công   ty     công   nghệ   hàng   đầu    Việt   Nam    .     
PhoBERT:   O      O      B-ORG  I-ORG   O      O      O      O      O      O      O      O      O      O      B-LOC  I-LOC  O     
Finetuned: B-ORG  I-ORG  I-ORG  I-ORG 

In [16]:
for tokens, l, l_ in zip(token_list, label_list, label_list_):
    print(tokens)
    print([label2id[i] for i in l_])
    print([label2id[i] for i in l])
    print("="*50)


['<s>', 'Thành', 'phố', 'Hồ', 'Chí', 'Minh', 'là', 'trung', 'tâm', 'kinh', 'tế', 'lớn', 'nhất', 'Việt', 'Nam', '.', '</s>']
[0, 5, 6, 6, 6, 6, 0, 0, 6, 6, 6, 0, 0, 5, 6, 0, 0]
[0, 5, 6, 6, 6, 6, 0, 0, 6, 6, 0, 0, 0, 5, 6, 0, 0]
['<s>', 'Tiến', 'sĩ', 'Lê', 'Thị', 'Thu', 'Hà', 'đang', 'nghiên', 'cứu', 'về', 'trí', 'tu@@', 'ệ', 'nhân', 'tạo', '.', '</s>']
[0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['<s>', 'Tập', 'đoàn', 'FPT', 'Software', 'là', 'một', 'trong', 'những', 'công', 'ty', 'công', 'nghệ', 'hàng', 'đầu', 'Việt', 'Nam', '.', '</s>']
[0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0]
[0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 4, 4, 4, 0, 0, 5, 6, 0, 0]
['<s>', 'Vịnh', 'Hạ', 'Long', 'được', 'UNESCO', 'công', 'nhận', 'là', 'Di', 'sản', 'Thiên', 'nhiên', 'Thế', 'giới', '.', '</s>']
[0, 5, 6, 6, 0, 3, 0, 0, 0, 3, 4, 4, 4, 4, 4, 0, 0]
[0, 5, 6, 6, 0, 3, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 0]
['<s>', 'Công', 'ty', 'cổ', 'phần', 'Sữa

In [11]:
trainer.save_model("./phobert-ner/final-model")
tokenizer.save_pretrained("./phobert-ner/final-model")


('./phobert-ner/final-model/tokenizer_config.json',
 './phobert-ner/final-model/special_tokens_map.json',
 './phobert-ner/final-model/vocab.txt',
 './phobert-ner/final-model/bpe.codes',
 './phobert-ner/final-model/added_tokens.json')